In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import sys
sys.path.append("/home/fxie/samza-azure-storage/milestone2/python")

In [11]:
form library.Benchmark2 import *

SyntaxError: invalid syntax (<ipython-input-11-d44f2812ab7f>, line 1)

In [ ]:
#bulk 50GB, overwrite 5GB, remove no cache because it's too slow
def test_data_disk_bulk_and_overwrite_with_compression():
    rocks_db_dir = "/home/fxie/rocksdb"
    for test_dir in ["/read_cache", "/readwrite_cache"]:
        print(f"test dir {test_dir}")
        benchmark2 = Benchmark2(rocks_db_dir = rocks_db_dir, test_dir = test_dir)
        benchmark2.para_map["COMPRESSION_TYPE"] = "snappy"
        benchmark2.para_map["CACHE_SIZE"] = "104857600"
        benchmark2.para_map["NUM_KEYS"] = "125000000"
        benchmark2.bulkload()
        
        #reduce kv num
        benchmark2.para_map["NUM_KEYS"] = "12500000"
        benchmark2.overwrite()
        benchmark2.clean_up()
        time.sleep(60)